<a href="https://colab.research.google.com/github/lettuceburger/AI_Trials/blob/main/pinecone_multiple_pdf_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai langchain  tiktoken pypdf unstructured[local-inference] gradio chromadb


In [ ]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone, Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ['OPENAI_API_KEY'] =""

In [ ]:
#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader('C:/Users/ASUS/OneDrive/Documents/learn/', glob="**/*.pdf")
readme_loader = DirectoryLoader('C:/Users/ASUS/OneDrive/Documents/learn/', glob="**/*.md")
txt_loader = DirectoryLoader('C:/Users/ASUS/OneDrive/Documents/learn/', glob="**/*.txt")

In [ ]:
#take all the loader
loaders = [pdf_loader, readme_loader, txt_loader]

#lets create document 
documents = []
for loader in loaders:
    documents.extend(loader.load())

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [ ]:
print (f'You have {len(documents)} document(s) in your data')
print (f'There are {len(documents[0].page_content)} characters in your document')

You have 3 document(s) in your data
There are 272002 characters in your document


In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=40) #chunk overlap seems to work better
documents = text_splitter.split_documents(documents)
print(len(documents))

Created a chunk of size 1183, which is longer than the specified 1000
Created a chunk of size 1104, which is longer than the specified 1000
Created a chunk of size 1015, which is longer than the specified 1000
Created a chunk of size 1124, which is longer than the specified 1000
Created a chunk of size 1006, which is longer than the specified 1000
Created a chunk of size 1106, which is longer than the specified 1000
Created a chunk of size 1015, which is longer than the specified 1000
Created a chunk of size 1213, which is longer than the specified 1000
Created a chunk of size 1045, which is longer than the specified 1000
Created a chunk of size 1150, which is longer than the specified 1000
Created a chunk of size 1106, which is longer than the specified 1000
Created a chunk of size 1194, which is longer than the specified 1000
Created a chunk of size 1298, which is longer than the specified 1000
Created a chunk of size 2582, which is longer than the specified 1000
Created a chunk of s

855


In [ ]:
documents[0]

Document(page_content='Global Business Driven\n\nHR Transformation\n\nThe Journey Continues\n\nPreface\n\nSee farther down the line\n\nYou can turn to the TV talking heads to hear opinions about the future. Or you can turn to the people in the trenches for an informed view of what really lies down the track.\n\nGlobal Business Driven HR Transformation is a crosscurrent of rapid changes in culture, technology, law, and leading practices. There’s no such thing as an organization that’s arrived at its destination — there are only ones that keep evolving and ones content to stand still. In our new book, Global Business Driven HR Transformation: The Journey Continues, Deloitte has mined the experience and vision of its leading field professionals to explore the many future trends that are shaping the way companies relate to their people.', metadata={'source': 'C:\\Users\\ASUS\\OneDrive\\Documents\\learn\\global-business-driven-hr-transformation.pdf'})

In [ ]:
documents[40]

Document(page_content='In the years to come, a global organization will likely be one that allows different HR practices in different places to the extent that they benefit the organization. When operations in different parts of the world each operate according to local needs, conditions, and opportunities — and when they have common systems and standards that let them share relevant information and data — then they will be the parts that make a global organization truly whole.\n\nGlobal Business Driven HR Transformation The Journey Continues   15\n\n2\n\n3\n\nSolution integration\n\nChange analytics\n\n1\n\nWhat is truly\n\nmeant by global\n\nSetting the Global HR\n\nTransformation Strategy\n\n2\n\nSolution Integration', metadata={'source': 'C:\\Users\\ASUS\\OneDrive\\Documents\\learn\\global-business-driven-hr-transformation.pdf'})

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
!pip install pinecone-client -q

In [ ]:
import os
import getpass
PINECONE_API_KEY = getpass.getpass('Pinecone API Key:')

Pinecone API Key:········


In [ ]:
PINECONE_ENV = getpass.getpass('Pinecone Environment:')

Pinecone Environment:········


In [ ]:
import pinecone 

# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV  # next to api key in console
)

index_name = "langchain-demo"

vectorstore = Pinecone.from_documents(documents, embeddings, index_name=index_name)

C:\Users\ASUS\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
query = "What is MRF?"
docs = vectorstore.similarity_search(query)

In [ ]:
len(docs) #it went on and search on the 4 different vectors to find the similarity

4

In [ ]:
print(docs[0].page_content)

Karvy Financial Services Ltd

iii. Manpower Requirement form (MRF): All hiring requests have to be channeled through MRF. This would help us track and reconcile the category of hiring under backfill or additional hire. The idea is not be bureaucratic but to have control over the process. The MRF has to be sent to KFSL HR with pre approval from the CEO.

v. Sourcing of candidates: On receipt of the MRF, KFSL HR can make use of any of the below mentioned sources as per the requirement.

Job Portals – Job portals will be made available to the sourcing team. This is a very good and

cost effective source for hiring of middle level positions.

Internal  referral  –  All  hiring  requirements  would  be  published  for  the  information  of  all

employees  of  the  company  (exceptions  can  be  made  if  the  KFSL  HR  decides  not  to  share  the

requirements internally for confidential reasons). Internal referral should be made a strong source


In [ ]:
from langchain.llms import OpenAI

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":2})
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), retriever)

In [ ]:
#chat_history = []
#query = "What are the grades and level in Karvy?"
#result = qa({"question": query, "chat_history": chat_history})
#result["answer"]

In [ ]:
#chat_history.append((query, result["answer"]))
#chat_history

In [ ]:
#query = "which grade is associated with level 1 in Karvy?"
#result = qa({"question": query, "chat_history": chat_history})
#result["answer"]

In [ ]:
from IPython.display import display
import ipywidgets as widgets

In [ ]:
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""
    
    if query.lower() == 'exit':
        print("Thanks for the chat!")
        return
    
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))
    
    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="Orange">Chatbot:</font></b> {result["answer"]}'))

print("Chat with your data. Type 'exit' to stop")

input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Chat with your data. Type 'exit' to stop


Text(value='', placeholder='Please enter your question:')

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


HTML(value='<b>User:</b> what is MRF')

HTML(value='<b><font color="Orange">Chatbot:</font></b>  MRF stands for Manpower Requirement form. It is a for…

...

HTML(value='<b><font color="Orange">Chatbot:</font></b>  The grades and levels in Karvy Financial Services Ltd…

HTML(value='<b>User:</b> can you present the above data in tabular format')

HTML(value='<b><font color="Orange">Chatbot:</font></b>  Level 8: Vice President, Level 7: Assistant Vice Pres…

HTML(value='<b>User:</b> THE PROCESS FOR CONDUCTING INQUIRY in IIMA')

HTML(value='<b><font color="Orange">Chatbot:</font></b>  Upon receipt of the complaint, the Chairperson/Presid…

In [ ]:
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    
    def respond(user_message, chat_history):
        print(user_message)
        print(chat_history)
        # Get response from QA chain
        response = qa({"question": user_message, "chat_history": chat_history})
        # Append user message and response to chat history
        chat_history.append((user_message, response["answer"]))
        print(chat_history)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True, share=True)